Kütüphaneleri **import** ettik.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')


**flights** isimli csv dosyasından veriyi çektik.

In [ ]:
df = pd.read_csv('/kaggle/input/flights/flights.csv')

sütunlara ait verilerin formatlarını inceledik.

In [ ]:
df.info()

İlk 5 satırın verilerini önizleme yaptık.

In [ ]:
df.head()

***time_hour*** isimli sütundaki veriler tarih olduğu için **Object** veri tipini ***date_time*** formatına dönüştürdük.

In [ ]:
df['time_hour'] = pd.to_datetime(df['time_hour'])


In [ ]:
df.head()

In [ ]:
# Zaman sütunlarını datetime'a çevir
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Haftanın günü (0 = Pazartesi, 6 = Pazar)
df['day_of_week'] = df['date'].dt.dayofweek

# Hafta sonu mu? (Cumartesi & Pazar)
df['weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Ayların döngüsel dönüşümü (Mevsimsellik için)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# Kalkış saati bilgisi (sched_dep_time float/int, örn: 540 → 5:40)
df['sched_dep_hour'] = df['sched_dep_time'] // 100
df['sched_dep_minute'] = df['sched_dep_time'] % 100


In [ ]:
# Ortalama hız (mil/saat) → distance / air_time * 60
df['avg_speed'] = (df['distance'] / df['air_time']) * 60


In [ ]:
# Uçuş yönü (örneğin 'JFK_MIA')
df['route'] = df['origin'] + "_" + df['dest']


In [ ]:
# Taşıyıcı bazında ortalama kalkış gecikmesi
carrier_delay_mean = df.groupby('carrier')['dep_delay'].mean().to_dict()
df['carrier_avg_delay'] = df['carrier'].map(carrier_delay_mean)


Son 5 satıra ait verileri önizledim.

In [ ]:
df.tail()

Her sütunda ne kadar boş veri sayısı var gözlemledik.

In [ ]:
df.isnull().sum()

her sütunda ne kadar boş veri var sayısını görmek için isnull kullanıldı.

In [ ]:
df.tail()

Boş hücreleri bir önceki hücrede yer alan verilerle doldurduk.

In [ ]:
df['dep_time'] = df['dep_time'].ffill()
df['dep_delay'] = df['dep_delay'].ffill()
df['arr_time'] = df['arr_time'].ffill()
df['arr_delay'] = df['arr_delay'].ffill()
df['tailnum'] = df['tailnum'].ffill()
df['air_time'] = df['air_time'].ffill()
df['avg_speed'] = df['avg_speed'].ffill()

Tekrar hücredeki boş hücrelerin sayısını gözlemledik.

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

# **Aylara Göre Uçuş Sayısı**

In [ ]:
print(df.columns)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x="month", data=df, palette="viridis")
plt.title("Aylara Göre Uçuş Sayısı")
plt.xlabel("Ay")
plt.ylabel("Uçuş Sayısı")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# **Havayolu Şirketlerine Göre Uçuş Sayısı**

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(y="carrier", data=df, order=df["carrier"].value_counts().index, palette="magma")
plt.title("Havayolu Şirketlerine Göre Uçuş Sayısı")
plt.xlabel("Uçuş Sayısı")
plt.ylabel("Havayolu")
plt.tight_layout()
plt.show()


# **Gecikmeli Uçuşların Dağılımı (Kalkış Gecikmesi)**

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df["dep_delay"].dropna(), bins=50, kde=True, color="red")
plt.title("Kalkış Gecikmesi Dağılımı")
plt.xlabel("Gecikme (Dakika)")
plt.ylabel("Uçuş Sayısı")
plt.xlim(-20, 200)  # Aykırı değerleri dışarda bırakmak için
plt.tight_layout()
plt.show()

# **Gün Bazında Ortalama Gecikmeler**

In [ ]:
daily_delay = df.groupby("day")["dep_delay"].mean()

plt.figure(figsize=(10, 6))
sns.lineplot(x=daily_delay.index, y=daily_delay.values, marker="o")
plt.title("Günlere Göre Ortalama Kalkış Gecikmeleri")
plt.xlabel("Gün")
plt.ylabel("Ortalama Gecikme (dk)")
plt.grid(True)
plt.tight_layout()
plt.show()

Model denemek için kütüphaneleri import ettik.

In [ ]:
# Klasik model: Random Forest
from sklearn.ensemble import RandomForestRegressor  # Birden çok karar ağacına dayalı regresyon modeli

# Alternatif modeller:
import xgboost as xgb  # XGBoost: Hızlı ve etkili bir boosting algoritması
from sklearn.ensemble import GradientBoostingRegressor  # sklearn içerisindeki boosting algoritması
from lightgbm import LGBMRegressor  # LightGBM: Büyük veri setlerinde verimli çalışan boosting algoritması

In [ ]:
# Eğitim verisini Ocak-Eylül olarak ayırıyoruz:
train_df = df[df['month'] <= 9].copy()

# Test verisini Ekim-Aralık olarak ayırıyoruz:
test_df = df[df['month'] > 9].copy()

# Hedef ve özellikleri belirledik
target = 'dep_delay'
features = ['month', 'day', 'hour', 'origin', 'dest', 'distance', 'air_time']

# Kategorik değişkenler var ise get_dummies ile dönüştürelim:
train_X = pd.get_dummies(train_df[features])
test_X = pd.get_dummies(test_df[features])

# Eğitim ve test verilerindeki sütunları eşitleyelim:
train_X, test_X = train_X.align(test_X, join='left', axis=1, fill_value=0)
train_y = train_df[target]


# **3. Model Eğitimi ve Tahmin** 
**a. RandomForestRegressor (Referans Model)** 

In [ ]:
# Keras'tan gerekli modülleri içe aktar (Sequential model ve Dense katmanı)
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense

# Boş bir Sequential model oluştur
model = Sequential()

# İlk gizli katman: 128 nöron, giriş boyutu train_X'in özellik sayısı kadar, aktivasyon fonksiyonu ReLU
model.add(Dense(128, input_dim=train_X.shape[1], activation='relu'))

# İkinci gizli katman: 64 nöron, aktivasyon fonksiyonu ReLU
model.add(Dense(64, activation='relu'))

# Çıkış katmanı: 1 nöron (tek bir sayısal değer tahmini için)
model.add(Dense(1))  # çıkış katmanı

# Modeli derle: Kayıp fonksiyonu olarak Ortalama Mutlak Hata (MAE), optimizasyon için Adam kullan
model.compile(loss='mean_absolute_error', optimizer='adam')

# Modeli eğit: eğitim verisi, 10 epoch, batch size 64
model.fit(train_X, train_y, epochs=10, batch_size=64)

# Test verisi üzerinde tahmin yap ve sonuçları test_df'ye 'predicted_nn' sütunu olarak ekle
test_df['predicted_nn'] = model.predict(test_X).flatten()

# Gerçek test değerleri ile tahmin edilen değerler arasındaki MAE'yi hesapla
mae_nn = mean_absolute_error(test_df['dep_delay'], test_df['predicted_nn'])

# Sinir ağı modelinin MAE değerini ekrana yazdır
print("NN MAE:", mae_nn)


In [ ]:

import pandas as pd
import lightgbm as lgb

# 'year', 'month' ve 'day' sütunlarından datetime formatında yeni 'date' sütunu oluştur
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Eğitim verisini 30 Eylül 2013 tarihine kadar olan verilerle oluştur
train_df = df[df['date'] <= '2013-09-30']

# Test verisini 1 Ekim 2013 ile 31 Aralık 2013 tarihleri arasındaki verilerle oluştur
test_df = df[(df['date'] > '2013-09-30') & (df['date'] <= '2013-12-31')]

# Modelde kullanılacak özellikler (bağımsız değişkenler)
features = ['month', 'day', 'dep_time', 'sched_dep_time', 'hour', 'minute']

# Tahmin edilmek istenen hedef değişken (bağımlı değişken)
target = 'dep_delay'

# Eğitim verisindeki özellikleri ve hedefi ayır
train_X = train_df[features]
train_y = train_df[target]

# Test verisindeki özellikler
test_X = test_df[features]

# LightGBM regresyon modelini oluştur (rastgelelik sabitlenmiş)
lgbm_model = lgb.LGBMRegressor(random_state=42)

# Modeli eğitim verisi ile eğit
lgbm_model.fit(train_X, train_y)

# Test veri çerçevesinin kopyasını oluştur (orijinal veriyi değiştirmemek için)
test_df = test_df.copy()

# Test verisi üzerinde tahmin yap ve sonuçları yeni bir sütun olarak ekle
test_df.loc[:, 'predicted_dep_delay'] = lgbm_model.predict(test_X)

# Tarih, gerçek gecikme süresi ve tahmin edilen gecikme süresini göster
print(test_df[['date', 'dep_delay', 'predicted_dep_delay']].head())



In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Gerçek değerler (test setindeki gerçek dep_delay sütunu)
y_true = test_df['dep_delay']

# Model tarafından tahmin edilen değerler
y_pred = test_df['predicted_dep_delay']

# Ortalama Mutlak Hata (MAE) hesapla — tahminlerin gerçeklerden ortalama ne kadar saptığını ölçer
mae = mean_absolute_error(y_true, y_pred)

# Ortalama Kare Hata (MSE) kökünü (RMSE) hesapla — hata sapmalarının karekökü, daha büyük hatalara daha fazla ceza verir
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

# MAE değerini ekrana yazdır
print(f"MAE: {mae}")

# RMSE değerini ekrana yazdır
print(f"RMSE: {rmse}")


# **1. Random Forest Regressor**

In [ ]:
# Random Forest regresyon modeli için gerekli kütüphaneleri içe aktar
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Random Forest modelini oluştur (100 ağaç, rastgelelik için sabit tohum)
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)

# Modeli eğitim verisi üzerinde eğit
rf_model.fit(train_X, train_y)

# Test verisi üzerinde tahmin yap
rf_pred = rf_model.predict(test_X)

# Test verisindeki gerçek değerlerle model tahminleri arasındaki Ortalama Mutlak Hata'yı hesapla ve yazdır
print("Random Forest MAE:", mean_absolute_error(test_df['dep_delay'], rf_pred))

# Test verisindeki gerçek değerlerle model tahminleri arasındaki Karekök Ortalama Kare Hata'yı (RMSE) hesapla ve yazdır
print("Random Forest RMSE:", np.sqrt(mean_squared_error(test_df['dep_delay'], rf_pred)))



# **2. XGBoost Regressor**

In [ ]:
# XGBoost kütüphanesini içe aktar
import xgboost as xgb

# XGBoost regresyon modelini oluştur (100 ağaç, rastgelelik sabit)
xgb_model = xgb.XGBRegressor(random_state=42, n_estimators=100)

# Modeli eğitim verisi üzerinde eğit
xgb_model.fit(train_X, train_y)

# Test verisi üzerinde tahmin yap
xgb_pred = xgb_model.predict(test_X)

# Gerçek değerlerle tahmin edilen değerler arasındaki Ortalama Mutlak Hata'yı (MAE) hesapla ve yazdır
print("XGBoost MAE:", mean_absolute_error(test_df['dep_delay'], xgb_pred))

# Gerçek değerlerle tahmin edilen değerler arasındaki Karekök Ortalama Kare Hata'yı (RMSE) hesapla ve yazdır
print("XGBoost RMSE:", np.sqrt(mean_squared_error(test_df['dep_delay'], xgb_pred)))



# **3. CatBoost Regressor**

In [ ]:
# CatBoost kütüphanesinden CatBoostRegressor sınıfını içe aktar
from catboost import CatBoostRegressor

# CatBoost regresyon modelini oluştur (100 iterasyon, sessiz modda, rastgelelik sabit)
cat_model = CatBoostRegressor(random_state=42, verbose=0, iterations=100)

# Modeli eğitim verisi üzerinde eğit
cat_model.fit(train_X, train_y)

# Test verisi üzerinde tahmin yap
cat_pred = cat_model.predict(test_X)

# Gerçek ve tahmin edilen değerler arasındaki Ortalama Mutlak Hata'yı (MAE) hesapla ve yazdır
print("CatBoost MAE:", mean_absolute_error(test_df['dep_delay'], cat_pred))

# Gerçek ve tahmin edilen değerler arasındaki Karekök Ortalama Kare Hata'yı (RMSE) hesapla ve yazdır
print("CatBoost RMSE:", np.sqrt(mean_squared_error(test_df['dep_delay'], cat_pred)))


# **4. Gradient Boosting Regressor (sklearn)**

In [ ]:
# Sklearn'den GradientBoostingRegressor sınıfını içe aktar
from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting regresyon modelini oluştur (100 ağaç, rastgelelik sabit)
gb_model = GradientBoostingRegressor(random_state=42, n_estimators=100)

# Modeli eğitim verisi üzerinde eğit
gb_model.fit(train_X, train_y)

# Test verisi üzerinde tahmin yap
gb_pred = gb_model.predict(test_X)

# Gerçek değerler ile tahmin edilen değerler arasındaki Ortalama Mutlak Hata (MAE) hesapla ve yazdır
print("Gradient Boosting MAE:", mean_absolute_error(test_df['dep_delay'], gb_pred))

# Gerçek değerler ile tahmin edilen değerler arasındaki Karekök Ortalama Kare Hata (RMSE) hesapla ve yazdır
print("Gradient Boosting RMSE:", np.sqrt(mean_squared_error(test_df['dep_delay'], gb_pred)))



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Random Forest modelini oluştur (rastgelelik sabit)
rf = RandomForestRegressor(random_state=42)

# Modelde denenmek istenen parametrelerin aralıkları/dizileri
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],      # Ağaç sayısı
    'max_depth': [None, 10, 20, 30, 40, 50],       # Maksimum derinlik
    'min_samples_split': [2, 5, 10],                # İç düğüm bölünmesi için minimum örnek sayısı
    'min_samples_leaf': [1, 2, 4],                  # Yaprak düğümde minimum örnek sayısı
    'max_features': ['auto', 'sqrt', 'log2']        # Özellik seçimi yöntemi
}

# RandomizedSearchCV ile 20 farklı parametre kombinasyonunu 3 katlı çapraz doğrulama ile dene
random_search = RandomizedSearchCV(
    estimator=rf,                     # Model
    param_distributions=param_dist,  # Parametre aralıkları
    n_iter=20,                       # Kaç farklı parametre kombinasyonu denenecek
    cv=3,                            # 3 katlı çapraz doğrulama
    verbose=2,                      # Çalışma sırasında çıktı detay seviyesi
    random_state=42,                 # Rastgelelik sabit
    n_jobs=-1,                      # Tüm işlemciler kullanılacak
    scoring='neg_mean_absolute_error'  # Optimize edilecek metrik (negatif MAE)
)

# Parametre araması ile modeli eğit
random_search.fit(train_X, train_y)

# En iyi bulunan parametreleri yazdır
print("Best parameters:", random_search.best_params_)

# En iyi modeli al
best_rf = random_search.best_estimator_

# Test verisi üzerinde tahmin yap
rf_pred = best_rf.predict(test_X)

# Test seti için Ortalama Mutlak Hata hesapla
mae = mean_absolute_error(test_df['dep_delay'], rf_pred)

# Test seti için Karekök Ortalama Kare Hata hesapla
rmse = np.sqrt(mean_squared_error(test_df['dep_delay'], rf_pred))

# Performans sonuçlarını yazdır
print(f"Optimized Random Forest MAE: {mae}")
print(f"Optimized Random Forest RMSE: {rmse}")



In [ ]:
# Sadece 1 Ekim - 31 Aralık arası filtreleme
tahminler = test_df[(test_df['date'] >= '2013-10-01') & (test_df['date'] <= '2013-12-31')]

# Gerekli sütunları göster (gözlem + tahmin)
print(tahminler[['date', 'dep_delay', 'predicted_dep_delay']].head(20))  # ilk 20 satır
